In [1]:
from datetime import datetime
import xarray as xr

In [2]:
fp = '../dev_tools/test_files/EXAMPLE_SHIMADA_202405.nc'

In [3]:
ds = xr.open_dataset(fp)

In [6]:
ds = ds.sel(time = slice(None, datetime(2024,6,3,4,15,0)))

In [9]:
ds.to_netcdf('TEST_SHIMADA_202405.nc')